In [1]:
import random
import gym
import numpy as np
from collections import deque
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.optimizers import Adam

ENV_NAME = "CartPole-v0"

GAMMA = 0.95
LEARNING_RATE = 0.001

MEMORY_SIZE = 1000000
BATCH_SIZE = 20

EXPLORATION_MAX = 1.0
EXPLORATION_MIN = 0.01
EXPLORATION_DECAY = 0.995


class DQNSolver:

    def __init__(self, observation_space, action_space):
        self.exploration_rate = EXPLORATION_MAX

        self.action_space = action_space
        self.memory = deque(maxlen=MEMORY_SIZE)

        self.model = Sequential()
        self.model.add(Dense(24, input_shape=(observation_space,), activation="relu"))
        self.model.add(Dense(24, activation="relu"))
        self.model.add(Dense(self.action_space, activation="linear"))
        self.model.compile(loss="mse", optimizer=Adam(lr=LEARNING_RATE))

    def remember(self, state, action, reward, next_state, done):
        self.memory.append((state, action, reward, next_state, done))

    def act(self, state):
        if np.random.rand() < self.exploration_rate:
            return random.randrange(self.action_space)
        q_values = self.model.predict(state)
        return np.argmax(q_values[0])

    def experience_replay(self):
        if len(self.memory) < BATCH_SIZE:
            return
        batch = random.sample(self.memory, BATCH_SIZE)
        for state, action, reward, state_next, terminal in batch:
            q_update = reward
            if not terminal:
                q_update = (reward + GAMMA * np.amax(self.model.predict(state_next)[0]))
            q_values = self.model.predict(state)
            q_values[0][action] = q_update
            self.model.fit(state, q_values, verbose=0)
        self.exploration_rate *= EXPLORATION_DECAY
        self.exploration_rate = max(EXPLORATION_MIN, self.exploration_rate)


def cartpole():
    env = gym.make(ENV_NAME)
    observation_space = env.observation_space.shape[0]
    action_space = env.action_space.n
    dqn_solver = DQNSolver(observation_space, action_space)
    run = 0
    while True:
        run += 1
        state = env.reset()
        state = np.reshape(state, [1, observation_space])
        step = 0
        while True:
            step += 1
            action = dqn_solver.act(state)
            state_next, reward, terminal, info = env.step(action)
            state_next = np.reshape(state_next, [1, observation_space])
            dqn_solver.remember(state, action, reward, state_next, terminal)
            state = state_next
            dqn_solver.experience_replay()
            if terminal:
                print("Run: " + str(run) + ", exploration: " + str(dqn_solver.exploration_rate) + ", score: " + str(step))
                break


if __name__ == "__main__":
    cartpole()

Run: 1, exploration: 1.0, score: 15
Run: 2, exploration: 0.9369146928798039, score: 17
Run: 3, exploration: 0.810157377815473, score: 29
Run: 4, exploration: 0.6935613678313175, score: 31
Run: 5, exploration: 0.6465587967553006, score: 14
Run: 6, exploration: 0.5704072587541458, score: 25
Run: 7, exploration: 0.5238143793828016, score: 17
Run: 8, exploration: 0.457510005540005, score: 27
Run: 9, exploration: 0.42437208406280985, score: 15
Run: 10, exploration: 0.29580711868545667, score: 72
Run: 11, exploration: 0.2494556624678441, score: 34
Run: 12, exploration: 0.23255008201124722, score: 14
Run: 13, exploration: 0.1712870076899825, score: 61
Run: 14, exploration: 0.1451730787173275, score: 33
Run: 15, exploration: 0.12304040492325048, score: 33
Run: 16, exploration: 0.10480604571960442, score: 32
Run: 17, exploration: 0.09386293109673778, score: 22
Run: 18, exploration: 0.08157186144027828, score: 28
Run: 19, exploration: 0.06477420436570952, score: 46
Run: 20, exploration: 0.054079

KeyboardInterrupt: 